# Assignment

- 제출자: 20기 황태연
- 제출 일자: 2023.08.08. (화)

## 우리의 목표 : 나이브베이즈 문제 해결하기
1.1) 입력문서가 {fast, furious, fun} 만을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류되는가?

##### 노트북 파일을 따라오면서, 빈칸과 질문에 대한 물음을 모두 채우시면 과제 완료 입니다 

### 주어진 dataset 을 확인해 보기위해 엑셀 파일을 읽습니다.

In [1]:
import numpy as np
import pandas as pd

In [2]:
documents = pd.read_excel('test_file.xlsx') 

In [3]:
documents

,label,message
0,comedy,"Fun couple, love love"
1,action,"fast Furious, shoot!!"
2,comedy,"couple^^, fly, fast, fun, fun"
3,action,"furious, shoot shoot fun"
4,action,"fly~~ fast shoot, Love"


####  Q1. 발견한 **문제점**(전처리가 필요한 부분)을 2가지 이상 적어주세요.
- 1. 만약 해당 데이터에서 분석하고자 하는 주요 단어가 fast, furious, fun뿐이라면 그 외의 단어는 제거해야 합니다.
- 2. 단어와 단어 사이의 띄어쓰기와 콤마(,)를 적절히 제거하여 단어만을 추출해야 합니다.
- 3. 대문자는 모두 소문자로 바꾸어서 단어의 횟수 파악에 용이하도록 해야 합니다.
- 4. 여러 특수문자들을 모두 제거해야 합니다.

### step 1. 단어를 모두 소문자로 바꾸어 줍니다. 
#### Q2. 이 과정이 왜 필요할까요?
- A. 대문자 소문자에 상관없이 같은 단어는 같은 것으로 분류하기 위함입니다. 예를 들어, Fun와 fun은 같은 단어로 취급해야 합니다.

#### Q3. 코드속 빈칸을 채워주세요

In [4]:
lower_documents = []
lower_documents = [d.lower() for d in documents['message']]
print(lower_documents)

['fun couple, love love', 'fast furious, shoot!!', 'couple^^, fly, fast, fun, fun', 'furious, shoot shoot fun', 'fly~~ fast shoot, love']


### step2. 특수문자를 없애줍니다.
#### string.punctuation 에는 특수문자들이 저장되어있는것을 확인할수 있습니다.

In [5]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Q4. string.punctuation 을 이용해서 특수문자를 제거해주세요.

In [6]:
import re

del_punctuation_documents = []

for i in lower_documents:
    del_punctuation_documents.append(re.sub(rf"[{re.escape(string.punctuation)}]", '', i))
    # string.punctuation을 정규표현식으로 나타내고, i 문장 내의 해당 기호를 빈문자열로 대체한다.
    
del_punctuation_documents

['fun couple love love',
 'fast furious shoot',
 'couple fly fast fun fun',
 'furious shoot shoot fun',
 'fly fast shoot love']

### step 3. 단어를 하나씩 띄어쓰기 단위로 쪼개어 줍니다.
#### Q5. 빈칸을 채워주세요.

In [7]:
preprocessed_documents = [[w for w in d.split()] for d in del_punctuation_documents]
preprocessed_documents

[['fun', 'couple', 'love', 'love'],
 ['fast', 'furious', 'shoot'],
 ['couple', 'fly', 'fast', 'fun', 'fun'],
 ['furious', 'shoot', 'shoot', 'fun'],
 ['fly', 'fast', 'shoot', 'love']]

### step 4. 각각의 단어가 문서에서 몇번 나왔는지 세봅시다.

In [8]:
frequency_list = []
from collections import Counter

frequency_list = [Counter(d) for d in preprocessed_documents]
frequency_list

[Counter({'fun': 1, 'couple': 1, 'love': 2}),
 Counter({'fast': 1, 'furious': 1, 'shoot': 1}),
 Counter({'couple': 1, 'fly': 1, 'fast': 1, 'fun': 2}),
 Counter({'furious': 1, 'shoot': 2, 'fun': 1}),
 Counter({'fly': 1, 'fast': 1, 'shoot': 1, 'love': 1})]

### step 5. 이제 우리가 하고싶은것은, 문자를 숫자로 변환하는 작업입니다!(그림 참고)
- 이를 위해 CountVectorizer를 사용합니다.
- CountVectorizer는 문서 집합에서 단어 토큰을 생성하고, 각 단어 수를 세어 BOW (Bag of Words) 인코딩한 벡터를 만들어줍니다.

- 문서에 해당단어가 몇번 포함되었는지를 나타낼 때 사용하는 방법입니다.

- 참고 : https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

![CountVectorized](https://user-images.githubusercontent.com/68625698/106378540-15d8ed80-63e9-11eb-8604-5c960c274867.PNG)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_vector.fit(documents['message'])

CountVectorizer()

In [10]:
doc_array = count_vector.fit_transform(documents['message']).toarray()
doc_array

array([[1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 2, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 2],
       [0, 1, 1, 0, 0, 1, 1]], dtype=int64)

In [19]:
count_vector.get_feature_names_out()
    # 이제는 get_feature_names를 지원하지 않고,
    # get_feature_names_out을 사용해야 합니다.

array(['couple', 'fast', 'fly', 'fun', 'furious', 'love', 'shoot'],
      dtype=object)

In [20]:
count_vector.vocabulary_

{'fun': 3,
 'couple': 0,
 'love': 5,
 'fast': 1,
 'furious': 4,
 'shoot': 6,
 'fly': 2}

In [21]:
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names_out())
frequency_matrix

,couple,fast,fly,fun,furious,love,shoot
0,1,0,0,1,0,2,0
1,0,1,0,0,1,0,1
2,1,1,1,2,0,0,0
3,0,0,0,1,1,0,2
4,0,1,1,0,0,1,1


In [22]:
frequency_matrix['count'] = frequency_matrix.sum(axis=1)

### step 6. 범주형 변수를 dummy변수로 변환해주는 작업(One-Hot Encoding!)을 해주어야합니다.
#### Q6. label 을 comedy =1, action =0 으로 변환해주세요

In [23]:
documents["label"] = pd.get_dummies(documents['label'])['comedy']

In [24]:
doc = pd.concat([documents['label'],frequency_matrix],axis=1)

In [25]:
doc

,label,couple,fast,fly,fun,furious,love,shoot,count
0,1,1,0,0,1,0,2,0,4
1,0,0,1,0,0,1,0,1,3
2,1,1,1,1,2,0,0,0,5
3,0,0,0,0,1,1,0,2,4
4,0,0,1,1,0,0,1,1,4


###  step 7. 나이브 베이즈 계산을 해봅시다!

#### Q7. 입력문서가 {fast, furious, fun} 을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류가 될까요? ( 계산과정을 채워주세요) 

In [26]:
doc = doc.values           # dataframe을 np-array로 변환해줍니다.
                         # return값은 np-matrix가 아니라 np-array입니다.
doc

array([[1, 1, 0, 0, 1, 0, 2, 0, 4],
       [0, 0, 1, 0, 0, 1, 0, 1, 3],
       [1, 1, 1, 1, 2, 0, 0, 0, 5],
       [0, 0, 0, 0, 1, 1, 0, 2, 4],
       [0, 0, 1, 1, 0, 0, 1, 1, 4]], dtype=int64)

In [27]:
type(doc)

numpy.ndarray

####  P(Y=comedy), P(Y=action) 계산하기

In [28]:
# P(Y=comedy)
p_comedy = sum(doc[:,0]==1) / len(doc)

# P(Y=action)
p_action = sum(doc[:,0]==0) / len(doc)

In [48]:
print('p_comedy : ', p_comedy)
print('p_action : ', p_action)

p_comedy :  0.4
p_action :  0.6


#### P(fast=1|comedy=1), P(furious=1|comedy=1), P(fun=1|comedy=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의

In [51]:
# 0:label 1:couple 2:fast 3:fly 4:fun 5:furious 6:love 7:shoot 8:count

# P(fast=1|comedy=1)
p_comedy_fast = sum((doc[:, 2]!=0) & (doc[:, 0]==1)) / sum(doc[:, 0]==1)

# P(furious=1|comedy=1)
p_comedy_furious = sum((doc[:, 5]!=0) & (doc[:, 0]==1)) / sum(doc[:, 0]==1) 

# P(fun=1|comedy=1)
p_comedy_fun = sum((doc[:, 4]!=0) & (doc[:, 0]==1)) / sum(doc[:, 0]==1)

In [52]:
print('p_comedy_fast : ' , p_comedy_fast)
print('p_comedy_furious : ' , p_comedy_furious)
print('p_comedy_fun : ' , p_comedy_fun)   

p_comedy_fast :  0.5
p_comedy_furious :  0.0
p_comedy_fun :  1.0


#### P(fast=1|action=1), P(furious=1|action=1), P(fun=1|action=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의

In [49]:
# P(fast=1|action=1)
p_action_fast = sum((doc[:, 2]!=0) & (doc[:, 0]==0)) / sum(doc[:, 0]==0)

# P(furious=1|action=1)
p_action_furious = sum((doc[:, 5]!=0) & (doc[:, 0]==0)) / sum(doc[:, 0]==0)

# P(fun=1|action=1)
p_action_fun = sum((doc[:, 4]!=0) & (doc[:, 0]==0)) / sum(doc[:, 0]==0)

In [50]:
print('p_action_fast : ' , p_action_fast)
print('p_action_furious : ' , p_action_furious)
print('p_action_fun : ' , p_action_fun)  

p_action_fast :  0.6666666666666666
p_action_furious :  0.6666666666666666
p_action_fun :  0.3333333333333333


#### P(Y = comedy| X = fast, furious, fun) , P(Y = action=1| X = fast, furious, fun) 값 계산하기

$$
P(Y=comedy|X=fast, furious, fun) = \frac{P(fast|comedy)P(furious|comedy)P(fun|comedy)P(comedy)}{P(X=fast, furious, fun)}
$$

In [58]:
p_fast_furious_fun = sum((doc[:, 2]!=0) & (doc[:, 4]!=0) & (doc[:, 5]!=0)) / sum(doc[:, 0]==1)
print(p_fast_furious_fun)

#P(Y = comedy| X = fast, furious, fun)
proba_comedy = (p_comedy_fast*p_comedy_furious*p_comedy_fun*p_comedy)/(p_fast_furious_fun)

#P(Y = action=1| X = fast, furious, fun)
proba_action = (p_action_fast*p_action_furious*p_action_fun*p_action)/(p_fast_furious_fun)

0.0


C:\Users\Poco\AppData\Local\Temp/ipykernel_3464/325066121.py:5: RuntimeWarning: invalid value encountered in double_scalars
  proba_comedy = (p_comedy_fast*p_comedy_furious*p_comedy_fun*p_comedy)/(p_fast_furious_fun)
C:\Users\Poco\AppData\Local\Temp/ipykernel_3464/325066121.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  proba_action = (p_action_fast*p_action_furious*p_action_fun*p_action)/(p_fast_furious_fun)


In [59]:
print('proba_comedy', proba_comedy)
print('proba_action', proba_action)

proba_comedy nan
proba_action inf


- 통계에서는 $P(X=fast, furious, fun)$는 상수로 취급하고 무시할 수 있다. (**Evidence**)

In [60]:
#P(Y = comedy| X = fast, furious, fun)
proba_comedy = (p_comedy_fast*p_comedy_furious*p_comedy_fun*p_comedy)

#P(Y = action=1| X = fast, furious, fun)
proba_action = (p_action_fast*p_action_furious*p_action_fun*p_action)

In [61]:
print('proba_comedy', proba_comedy)
print('proba_action', proba_action)

proba_comedy 0.0
proba_action 0.08888888888888888


### step 8. 다음 값을 비교해봅시다.
1. P(Y = comedy| X = fast, furious, fun)
2. P(Y = action=1| X = fast, furious, fun)

- 문서에 fast, furious, fun이 포함되어 있으면 action으로 분류될 가능성이 높다는 결론을 얻을 수 있습니다.

Q7. 어떤 문제점을 발견할수 있나요? 문제점을 해결하기 위한 방법으로는 어떤게 있을까요?
- A. Evidence 값이 0이므로 구하고자 하는 조건부확률을 구할 수 없습니다. 그러나 Evidence는 우리가 관심있는 Y와는 무관한 값이므로 상수로 취급하고 무시할 수 있습니다. 따라서 이를 무시함으로써 이 문제를 해결할 수 있습니다.

# DecisionTree Assignment - 20기 황태연

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

- 제출자: 20기 황태연
- 제출 일자: 2023.08.08. (화)

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

$$
Gini(D_i) = 1-\sum_j P_{j}^2
$$

In [42]:
def get_gini(df, label):
    # label에 들어있는 분류의 종류를 저장한다.
    classes = df[label].unique() # ['yes', 'no']
    
    # 데이터의 개수를 구한다.
    N = len(df)
    
    gini = 1
    # (P_j)**2를 gini = 1 에서 빼준다.
    for class_ in classes:
        gini -= (sum(df[label]==class_) / N)**2
        
    return gini

In [43]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [44]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    result = []
    for i in range(1, len(uniques)): # 1부터 len(uniques)-1까지
        for combination in combinations(uniques, i): # i개의 원소의 부분집합을 result에 추가
            result.append(list(combination))
    
    return result

In [45]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

- Gini index를 구하는 식은 다음과 같다.
$$
Gini(D) = \sum_i \frac{|D_i|}{|D|}Gini(D_i)
$$

In [47]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for key in binary_split:
        # df[attribute] 중 key 안에 있는 원소만 추출
        key_df = df[df[attribute].isin(key)]
        # df[attribute] 중 key 안에 없는 원소만 추출
        not_key_df = df[~df[attribute].isin(key)]
        
        D1 = len(key_df) # df[attribute] 중 key 안에 있는 원소의 개수
        D2 = n-D1 # df[attribute] 중 key 안에 없는 원소의 개수
        
        gini = 0
        
        gini_D1 = get_gini(key_df, label) # Gini(D_1)
        gini_D2 = get_gini(not_key_df, label) # Gini(D_2)
        
        gini = (D1/n)*gini_D1 + (D2/n)*gini_D2
        
        result[tuple(key)] =  gini
    return result

In [48]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [49]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [50]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [58]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    gini = min(get_attribute_gini_index(pd_data, feature, 'class_buys_computer').items())[1]
    print(f'Minimum Gini Index of {feature} : {gini:.4f}')

Minimum Gini Index of age : 0.3571
Minimum Gini Index of income : 0.4429
Minimum Gini Index of student : 0.3673
Minimum Gini Index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [59]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [72]:
from math import log2

def getEntropy(df, feature) :
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    # feature의 종류를 구합니다.
    features = df[feature].unique()
    
    # df의 데이터 개수를 구합니다.
    N = len(df)
    
    p = []
    # p_i를 구합니다.
    for f in features:
        n = sum(df[feature] == f)
        p.append(n/N)
    
    # Entropy를 계산합니다.
    p = np.array(p)
    H = -sum(p*np.log2(p))
    return H

In [73]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

- 각 변수의 엔트로피는 다음과 같이 구한다.
$$
E = \sum_i \frac{|S_i|}{|S|}H(S_I)
$$

- Information Gain은 다음과 같이 구한다.
$$
Gain = H(S) - E
$$

In [89]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}
    
    # 데이터의 개수를 구한다.
    N = len(df)
    
    # feature를 제외한 다른 변수들을 저장한다.
    other_features = df.drop(columns=[feature]).columns
    
    for f in other_features:
        # f 변수의 종류를 모두 구한다.
        s = df[f].unique()
        
        # 목표 변수의 Entropy를 구한 뒤 다른 변수의 Entropy를 빼준다.
        Entropy = getEntropy(df, feature)
        for s_i in s:
            # s_i에 해당하는 데이터만 담은 df를 생성한다.
            filtered_df = df[df[f] == s_i]
            
            # |s_i|를 구한다.
            n = len(filtered_df)
            
            # H(s_i)를 Entropy에 더한다.
            Entropy -= (n/N)*getEntropy(filtered_df, feature)
            
        result[f] = Entropy
    
    return result

In [90]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234164,
 'credit_rating': 0.048127030408269544}